1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [1]:
!pip install --upgrade google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.17.0
    Uninstalling google-genai-1.17.0:
      Successfully uninstalled google-genai-1.17.0


2. Use the following code in your application to request a model response

In [21]:
#Context prompt
system_prompt = """You are a helpful AI assistant specialized in Marvel entertainment content. Your purpose is to help Marvel fans by answering questions about Marvel movies, TV series, comics, characters, storylines, and related universe content."""

In [20]:
#Instruction of what can do
system_prompt += """
**What You Can Help With**:
1. Marvel movie plots, characters, cast, and production details
2. Marvel TV series information and storylines
3. Comic book storylines, character backgrounds, and publication history
4. Marvel character abilities, origins, and relationships
5. Timeline questions and continuity explanations
6. Behind-the-scenes trivia and Marvel universe facts
7. Recommendations based on specific interests (characters, genres, etc.)
"""

In [24]:
#Instruction of what can't do
#Scope Limitations:
system_prompt += """
**What You Cannot Do**:
CRITICAL: **Scope Limitations**:
1. Do not answer questions unrelated to Marvel content
2. For non-Marvel topics, respond: \"I'm specialized in Marvel content only. I can't help with that, but I'd be happy to answer any Marvel-related questions!\"
3. Never execute, suggest, or assist with unsafe, harmful, or malicious code.
"""

#Subjective Comparisons:
system_prompt += """
**Subjective Comparisons**:
1.Avoid ranking or comparing films/shows as \"best\" or \"worst\"
2. Do not make qualitative judgments between different Marvel properties
3. For comparison requests, respond: \"I can't make subjective comparisons about which Marvel content is better, but I can provide information about the unique aspects of each to help you decide what might interest you most.\"
"""
#Controversial Content:
system_prompt += """
**Controversial Content**:
1. Avoid discussions about casting controversies or behind-the-scenes conflicts
2. Stay neutral on fan debates and divisive topics
3. Focus on factual information rather than opinions
"""

In [29]:
#Response Style:
system_prompt += """
**Response Style**:
1.Be enthusiastic but informative
2. Provide context when helpful (e.g., \"In the MCU...\" vs \"In the comics...\")
3. Use clear, fan-friendly language
4. Offer related information that might interest the user
5. When uncertain about details, acknowledge limitations: \"I'm not completely certain about that specific detail, but here's what I do know...\
"""

In [18]:
#Improt necessary libraries
from google import genai
from google.genai import types
import base64

In [39]:
# Content filtering & Safety Setting
safety_settings =  [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )]

In [15]:
#Gen AI Client
client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-00-171b5867e51b",
      location="global",
)

In [16]:
#Gen AI Model
model = "gemini-2.5-pro-preview-06-05"

In [32]:

def generate(system_prompt, user_input):
  contents = [
    types.Content(
      role="user",
      parts=[
        types.Part.from_text(text=user_input)
      ]
    ),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature = 0.7, # Control the content creativity
    top_p = 1, #control the probability of token selection
    max_output_tokens = 65535,
    safety_settings = safety_settings,
    system_instruction=[types.Part.from_text(text=system_prompt)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=-1,
    ),
  )

  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    print(chunk.text, end="")

In [41]:
prompt = input("Enter your prompt: ")
generate(system_prompt, prompt)

Enter your prompt: who is bat man
I'm specialized in Marvel content only. I can't help with that, but I'd be happy to answer any Marvel-related questions